In [4]:
############################load img############################################
# load the val annotations file

import os



def get_annotations_map():
    valAnnotationsPath = './tiny-imagenet-200/val/val_annotations.txt'
    valAnnotationsFile = open(valAnnotationsPath, 'r')
    valAnnotationsContents = valAnnotationsFile.read()
    valAnnotations = {}

    for line in valAnnotationsContents.splitlines():
        pieces = line.strip().split()
        valAnnotations[pieces[0]] = pieces[1]
    
    return valAnnotations


#Sys
import numpy as np
from PIL import Image


def load_images(path,num_classes):
    #Load images
    
    print('Loading ' + str(num_classes) + ' classes')

    X_train0=np.zeros([num_classes*500,3,64,64],dtype='uint8')
    X_train=np.zeros([num_classes*500,64,64,3],dtype='uint8')
    y_train=np.zeros([num_classes*500], dtype='uint8')

    trainPath=path+'/train'

    print('loading training images...');

    i=0
    j=0
    annotations={}
    for sChild in os.listdir(trainPath):
        sChildPath = os.path.join(os.path.join(trainPath,sChild),'images')
        annotations[sChild]=j
        for c in os.listdir(sChildPath):
            X=np.array(Image.open(os.path.join(sChildPath,c)))
            if len(np.shape(X))==2:
                X_train0[i]=np.array([X,X,X])
            else:
                X_train0[i]=np.transpose(X,(2,0,1))
            X_train[i]=np.transpose(X_train0[i],(1,2,0))
            y_train[i]=j
            i+=1
        j+=1
        if (j >= num_classes):
            break

    #print('finished loading training images')

    val_annotations_map = get_annotations_map()

    X_test0 = np.zeros([num_classes*50,3,64,64],dtype='uint8')
    X_test = np.zeros([num_classes*50,64,64,3],dtype='uint8')
    y_test = np.zeros([num_classes*50], dtype='uint8')


    print('loading test images...')

    i = 0
    testPath=path+'/val/images'
    for sChild in os.listdir(testPath):
        if val_annotations_map[sChild] in annotations.keys():
            sChildPath = os.path.join(testPath, sChild)
            X=np.array(Image.open(sChildPath))
            if len(np.shape(X))==2:
                X_test0[i]=np.array([X,X,X])
            else:
                X_test0[i]=np.transpose(X,(2,0,1))
            X_test[i]=np.transpose( X_test0[i],(1,2,0)) 
            y_test[i]=annotations[val_annotations_map[sChild]]
            i+=1
        else:
            pass


   # print('finished loading test images')+str(i)

    return X_train,y_train,X_test,y_test

############################load img###################################################

# -*- coding: utf-8 -*-
"""ResNet50 model for Keras.

# Reference:

- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385)

Adapted from code contributed by BigMoyan.
"""

############################load pretrained model: ResNet152####################################################

# -*- coding: utf-8 -*-

import cv2
import copy

from keras.layers import Input, Dense, Conv2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Flatten, Activation, add
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras import initializers
from keras.engine import Layer, InputSpec
from keras import backend as K
from keras.layers import merge
from keras.layers import Dropout

import sys
sys.setrecursionlimit(3000)

class Scale(Layer):
    '''Custom Layer for ResNet used for BatchNormalization.
    
    Learns a set of weights and biases used for scaling the input data.
    the output consists simply in an element-wise multiplication of the input
    and a sum of a set of constants:
        out = in * gamma + beta,
    where 'gamma' and 'beta' are the weights and biases larned.
    # Arguments
        axis: integer, axis along which to normalize in mode 0. For instance,
            if your input tensor has shape (samples, channels, rows, cols),
            set axis to 1 to normalize per feature map (channels axis).
        momentum: momentum in the computation of the
            exponential average of the mean and standard deviation
            of the data, for feature-wise normalization.
        weights: Initialization weights.
            List of 2 Numpy arrays, with shapes:
            `[(input_shape,), (input_shape,)]`
        beta_init: name of initialization function for shift parameter
            (see [initializers](../initializers.md)), or alternatively,
            Theano/TensorFlow function to use for weights initialization.
            This parameter is only relevant if you don't pass a `weights` argument.
        gamma_init: name of initialization function for scale parameter (see
            [initializers](../initializers.md)), or alternatively,
            Theano/TensorFlow function to use for weights initialization.
            This parameter is only relevant if you don't pass a `weights` argument.
    '''
    def __init__(self, weights=None, axis=-1, momentum = 0.9, beta_init='zero', gamma_init='one', **kwargs):
        self.momentum = momentum
        self.axis = axis
        self.beta_init = initializers.get(beta_init)
        self.gamma_init = initializers.get(gamma_init)
        self.initial_weights = weights
        super(Scale, self).__init__(**kwargs)

    def build(self, input_shape):
        self.input_spec = [InputSpec(shape=input_shape)]
        shape = (int(input_shape[self.axis]),)

        self.gamma = K.variable(self.gamma_init(shape), name='%s_gamma'%self.name)
        self.beta = K.variable(self.beta_init(shape), name='%s_beta'%self.name)
        self.trainable_weights = [self.gamma, self.beta]

        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights

    def call(self, x, mask=None):
        input_shape = self.input_spec[0].shape
        broadcast_shape = [1] * len(input_shape)
        broadcast_shape[self.axis] = input_shape[self.axis]

        out = K.reshape(self.gamma, broadcast_shape) * x + K.reshape(self.beta, broadcast_shape)
        return out

    def get_config(self):
        config = {"momentum": self.momentum, "axis": self.axis}
        base_config = super(Scale, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

def identity_blockX(input_tensor, filters, stage, block):
    '''The identity_block is the block that has no conv layer at shortcut
    # Arguments
        input_tensor: input tensor
        kernel_size: defualt 3, the kernel size of middle conv layer at main path
        filters: list of integers, the nb_filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    '''
    eps = 1.1e-5
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    scale_name_base = 'scale' + str(stage) + block + '_branch'

    x = Conv2D(filters, (1, 1), name=conv_name_base + '2a', use_bias=False)(input_tensor)
    #x = Dropout(0.2, name=conv_name_base + '2a_dropout')(x)
   # x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2a')(x)
    #x = Scale(axis=bn_axis, name=scale_name_base + '2a')(x)
    x = Activation('relu', name=conv_name_base + '2a_relu')(x)

    x = Conv2D(filters, (1, 1), name=conv_name_base + '2b', use_bias=False)(x)
    #x = Dropout(0.2, name=conv_name_base + '2b_dropout')(x)
    #x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2b')(x)
    #x = Scale(axis=bn_axis, name=scale_name_base + '2b')(x)
    x = Activation('relu', name=conv_name_base + '2b_relu')(x)

    x = Conv2D(16, (1, 1), name=conv_name_base + '2c', use_bias=False)(x) #???
    #x = Dropout(0.2, name=conv_name_base + '2c_dropout')(x)
    #x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2c')(x)
    #x = Scale(axis=bn_axis, name=scale_name_base + '2c')(x)
    

    x = merge([x, input_tensor], mode='sum', name='res' + str(stage) + block)
    x = Activation('relu', name='res' + str(stage) + block + '_relu')(x)
    return x

def identity_blockY(input_tensor, filters, stage, block):
    '''The identity_block is the block that has no conv layer at shortcut
    # Arguments
        input_tensor: input tensor
        kernel_size: defualt 3, the kernel size of middle conv layer at main path
        filters: list of integers, the nb_filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    '''
    eps = 1.1e-5
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    scale_name_base = 'scale' + str(stage) + block + '_branch'

    
    y = Conv2D(filters, (3, 3), name=conv_name_base + '3a', use_bias=False)(input_tensor)
    #y = Dropout(0.2, name=conv_name_base + '3a_dropout')(y)
    #y = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '3a')(y)
    #y = Scale(axis=bn_axis, name=scale_name_base + '3a')(y)
    y = Activation('relu', name=conv_name_base + '3a_relu')(y)
    
    y = ZeroPadding2D((3, 3), name=conv_name_base + '3b_zeropadding')(y)

    y = Conv2D(filters, (3, 3), name=conv_name_base + '3c', use_bias=False)(y)
    #y = Dropout(0.2, name=conv_name_base + '3c_dropout')(y)
    #y = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '3c')(y)
    #y = Scale(axis=bn_axis, name=scale_name_base + '3c')(y)
    y = Activation('relu', name=conv_name_base + '3c_relu')(y)

    y = Conv2D(16, (3, 3), name=conv_name_base + '3d', use_bias=False)(y) #???
    #y = Dropout(0.2, name=conv_name_base + '3d_dropout')(y)
   # y = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '3d')(y)
    #y = Scale(axis=bn_axis, name=scale_name_base + '3d')(y)

    x = merge([y, input_tensor], mode='sum', name='res' + str(stage) + block)
    x = Activation('relu', name='res' + str(stage) + block + '_relu')(x)
    return x

def CrowdInceptron(include_top=False, weights=None,
             input_tensor=None, input_shape=None,
             pooling=None,
             classes=20):
    """Instantiates the ResNet50 architecture.

    """
    eps = 1.1e-5

    # Handle Dimension Ordering for different backends
    global bn_axis
    if K.image_dim_ordering() == 'tf':
        bn_axis = 3
        img_input = Input(shape=(64, 64, 3), name='data')
    else:
        bn_axis = 1
        img_input = Input(shape=(3, 64, 64), name='data')

    x = ZeroPadding2D((3, 3), name='conv1_zeropadding')(img_input)
    x = Conv2D(16, (5, 5), strides=(1, 1), name='conv1', use_bias=False)(x)
    #x = BatchNormalization(epsilon=eps, axis=bn_axis, name='bn_conv1')(x)
    #x = Scale(axis=bn_axis, name='scale_conv1')(x)
    x = Activation('relu', name='conv1_relu')(x)
    x = MaxPooling2D((3, 3), strides=(1, 1), name='pool1')(x)

    #x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_blockX(x, 16, stage=2, block='b')
    x = identity_blockY(x, 16, stage=2, block='c')

    #x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    x = identity_blockX(x, 16, stage=3, block='b')
    x = identity_blockY(x, 16, stage=3, block='c')
    

    #x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    x = identity_blockX(x, 16, stage=4, block='b')
    x = identity_blockY(x, 16, stage=4, block='c')
  
    #x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
    x = identity_blockX(x, 16, stage=5, block='b')
    x = identity_blockY(x, 16, stage=5, block='c')

    x = MaxPooling2D((5, 5), name='max_pool')(x)
    x = Flatten()(x)
    x = Dense(1024, activation='relu', name = 'fc00')(x)
    x = Dropout(0.5)(x)
    x = Dense(200, activation='softmax', name = 'fc200')(x) # old class!!

    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input
    # Create model.
    model = Model(inputs, x, name='resnet50')

    # load weights
    if weights == 'imagenet':
        if include_top:
            weights_path = get_file('resnet50_weights_tf_dim_ordering_tf_kernels.h5',
                                    WEIGHTS_PATH,
                                    cache_subdir='models',
                                    md5_hash='a7b3fe01876f51b976af0dea6bc144eb')
        else:
            weights_path = get_file('resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                    WEIGHTS_PATH_NO_TOP,
                                    cache_subdir='models',
                                    md5_hash='a268eb855778b3df3c7506639542a6af')
        model.load_weights(weights_path)
        if K.backend() == 'theano':
            layer_utils.convert_all_kernels_in_model(model)
            if include_top:
                maxpool = model.get_layer(name='avg_pool')
                shape = maxpool.output_shape[1:]
                dense = model.get_layer(name='fc1000')
                layer_utils.convert_dense_weights_data_format(dense, shape, 'channels_first')

        if K.image_data_format() == 'channels_first' and K.backend() == 'tensorflow':
            warnings.warn('You are using the TensorFlow backend, yet you '
                          'are using the Theano '
                          'image data format convention '
                          '(`image_data_format="channels_first"`). '
                          'For best performance, set '
                          '`image_data_format="channels_last"` in '
                          'your Keras config '
                          'at ~/.keras/keras.json.')
    elif weights is not None:
        model.load_weights(weights)

    return model
#########################################################################################

if __name__ == "__main__":
    
    import matplotlib.pyplot as plt
    
    #Keras
    import keras
    from keras.datasets import mnist
    from keras.models import Sequential
    from keras.layers import Dense, Dropout
    from keras.optimizers import RMSprop
    #from keras.regularizers import WeightRegularizer, ActivityRegularizer 
    from keras.layers.core import Dense, Dropout, Activation, Flatten
    from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
    from keras.layers.normalization import BatchNormalization 
    from keras.utils import np_utils
    from keras.optimizers import SGD, Adam
    from keras.preprocessing.image import ImageDataGenerator
    #from plotter import Plotter
    # from keras.utils.visualize_util import plot
    import h5py
    from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
    from keras.metrics import top_k_categorical_accuracy


    #Params
    num_classes = 200
    batch_size = 128 # cannot be larger...
    nb_epoch = 1

    h5f = h5py.File('IT.10.ad-200.vggs2.result.h5','r')#!!!
    X_train = h5f['xTrain'][:]
    Y_train =  h5f['yTrain'][:]
    X_test =  h5f['xTest'][:]
    Y_test =  h5f['yTest'][:]
    
    h5f.close()
    
    
    
    #print X dimension
    print('X_train shape:', X_train.shape)
    print(X_train.shape[0], 'train samples')
    print(X_test.shape[0], 'test samples')
    
    num_samples=len(X_train)

     
   
    
    print ('done')
    ##################################
    

    #input_shape = (64, 64, 3)

    model = CrowdInceptron()
    #loading weight from previous training
    model.load_weights('ep00.TI.ad-200.max.weights.best.hdf5')
    # compile the model (should be done *after* setting layers to non-trainable)
    #sgd = SGD(lr=1e-2, decay=1e-6, momentum=0.9, nesterov=True)
    #rmsprop = RMSprop(lr=0.001)
    #sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.5, nesterov=True)
    adam = Adam(lr = 1e-3 )
    
    #---------------------------
    
    #for top - 5 accuracy
    def top_5_accuracy(y_true, y_pred):
        return top_k_categorical_accuracy(y_true, y_pred, k=5)
    
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=[top_5_accuracy])
    
    
    
    
    
    #data augm
    datagen = ImageDataGenerator(
                #zca_whitening = True,
                rotation_range=1,
                #width_shift_range=0.05,
                #height_shift_range=0.05,
                #shear_range=0.2,
                #zoom_range=0.2,
                #channel_shift_range=0.2,
                #cval=0.4, very cumbersome
                horizontal_flip=True,
                vertical_flip=True,
                fill_mode='nearest'
              )
    datagen.fit(X_train)
    
    
    #checkpoint
    # checkpoint
    filepath="ep00.TI.ad-200.max2.weights.best.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
    
    reduceLr = ReduceLROnPlateau(monitor='loss', patience=50, mode= 'auto', factor = 0.5)
    callbacks_list = [checkpoint, reduceLr]
    
    #fit model
    history = model.fit_generator(datagen.flow(X_train.astype('float32')/255, Y_train, batch_size=batch_size),
                        steps_per_epoch=X_train.shape[0] // batch_size,
                        epochs=nb_epoch, shuffle=True, callbacks=callbacks_list,
                        validation_data=(X_test.astype('float32')/255, Y_test))


    
    #evaluate model
    score = model.evaluate(X_test.astype('float32')/255, Y_test, verbose=1)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])
    
    
    """
    #----------------load history----------------------------------
    h5f = h5py.File('IT.10.sgd1e-6.FU.reduce10.result.h5','r')
    hist_acc0 = h5f['acc'][:]
    hist_valAcc0 =  h5f['val_acc'][:]
    hist_loss0 =  h5f['loss'][:]
    hist_valLoss0 =  h5f['val_loss'][:]
    h5f.close()
    
    
    #--------------save history------------------------
    
    hist_acc = np.concatenate((hist_acc0, history.history['acc']), axis = 0)
    hist_valAcc = np.concatenate( (hist_valAcc0, history.history['val_acc']) , axis = 0)
    hist_loss = np.concatenate( ( hist_loss0, history.history['loss'])  , axis = 0)
    hist_valLoss =  np.concatenate( (hist_valLoss0, history.history['val_loss']) , axis = 0)
    
   
    hist_acc = history.history['acc']
    hist_valAcc = history.history['val_acc']
    hist_loss = history.history['loss']
    hist_valLoss = history.history['val_loss']
    
    #create h5
    h5f = h5py.File('IT.10.ad-200.max2.result.h5', 'w')
    h5f.create_dataset('acc', data=hist_acc)
    h5f.create_dataset('val_acc', data=hist_valAcc)
    h5f.create_dataset('loss', data=hist_loss)
    h5f.create_dataset('val_loss', data=hist_valLoss)
   
    
    
    h5f.close()
    
    
    #-------------plot-------------------
     #  "Accuracy"
    plt.plot(hist_acc )
    plt.plot(hist_valAcc)
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()
    
    # "Loss"
    plt.plot(hist_loss)
    plt.plot(hist_valLoss)
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()
    """

X_train shape: (100000, 64, 64, 3)
100000 train samples
10000 test samples
done


C:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:214: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
C:\Program Files\Anaconda3\lib\site-packages\keras\legacy\layers.py:464: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
C:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:252: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


Epoch 1/1
781/781 [==============================] - 482s 618ms/step - loss: 3.4007 - top_5_accuracy: 0.4967 - val_loss: 3.2653 - val_top_5_accuracy: 0.5296


C:\Program Files\Anaconda3\lib\site-packages\keras\callbacks.py:406: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


10000/10000 [==============================] - 4s 434us/step
Test score: 3.265316176223755
Test accuracy: 0.5296
